In [ ]:
import cv2
import time
import csv
import serial
import math
import random
import struct
from threading import Thread

class PsEye(Thread):
    def __init__(self):
        super().__init__()
        self._run = True
        self._video = cv2.VideoCapture(0)
        self._last_frame = None

    def get_act_frame(self):
        return self._last_frame

    def run(self):
        while self._run:
            check, frame = self._video.read()
            if check:
                self._last_frame = frame

    def save_last_frame(self, img_file_path, img_name):
        print('Saving img {}'.format(img_name))
        cv2.imwrite(str(img_file_path + img_name), self._last_frame)

    def end(self):
        self._run = False
        self.join()
        self._video.release()

    def frame_valid(self):
        return self._last_frame is not None

Camera = PsEye()
Camera.start()

# Waiting until camera ready
while not Camera.frame_valid():
    time.sleep(0.05)

print('Camera Running...')

# list available ports:
# $ python -m serial.tools.list_ports

ser = serial.Serial('/dev/ttyACM3',115200)
csv_file_path = "./image_labels.csv"
img_file_path = "./pictures/"


def move(random_rad):
    ser.write(str(random_rad).encode('utf-8'))

def get_label_value():
    ser.flushInput()
    value = ser.read_until()
    time.sleep(0.005)
    print(value)
    return float(value)

def take_picture(img_file_path, img_name):
    Camera.save_last_frame(img_file_path, img_name)
    
def log_csv(file, img_name, label):
    with open(file, 'a') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerow([img_name, label])
    

for a in range(20):  
    rand_increment = random.randint(100,200) 
    rand_rad = rand_increment/4096 * (2*math.pi)
    move(rand_rad)
    time.sleep(1.2)
    label = get_label_value()
    label =label %(2*math.pi)
    img_name = "{}_{}.png".format(label, time.time())
    take_picture(img_file_path, img_name)
    log_csv(csv_file_path, img_name, label)

Camera.end()
ser.close()

In [10]:
import cv2
import time
import csv
import serial
import math
import random
import struct
from threading import Thread


class Ser(Thread):
    def __init__(self, port):
        super().__init__()
        self._run = True
        self._ser = serial.Serial(port ,115200)
        self._last_value = 0.0
        self._valid = False

    def get_act_value(self):
        return self._last_value
    
    def value_valid(self):
        return self._valid
    
    def write(self, random_rad):
        self._ser.write(str(random_rad).encode('utf-8'))

    def run(self):
        print('Serial Thread running')
        while self._run:
            line = self._ser.read_until()
            #default reading: "b'1, 1.18/r/n"
            self._valid = bool(str(line).split(", ")[0].split("'")[1])
            #y=x[0]
            #z=y.split("'")
            #if (z[1] == 1):
            #    self._valid = True
            #else:
            #    self._valid = False
            self._last_value = float(str(line).split(", ")[1].split("\\")[0])
            #y=x[1]
            #z=y.split("\\")
            #self._last_value = float(z[0])
            #print(self._valid)
            #print(self._last_value)

    def end(self):
        self._run = False
        self.join()
        self._ser.close()


class PsEye(Thread):
    def __init__(self):
        super().__init__()
        self._run = True
        self._video = cv2.VideoCapture(0)
        self._last_frame = None

    def get_act_frame(self):
        return self._last_frame

    def run(self):
        while self._run:
            check, frame = self._video.read()
            if check:
                self._last_frame = frame

    def save_last_frame(self, img_file_path, img_name):
        print('Saving img {}'.format(img_name))
        cv2.imwrite(str(img_file_path + img_name), self._last_frame)

    def end(self):
        self._run = False
        self.join()
        self._video.release()

    def frame_valid(self):
        return self._last_frame is not None


#################################################################
# Programm starts here

# list available ports:
# $ python -m serial.tools.list_ports

csv_file_path = "./image_labels.csv"
img_file_path = "./pictures/"

num_images = 5

port = '/dev/ttyACM1'

print("Starting Serial Thread...")
ser = Ser(port)
ser.start()
time.sleep(1)

camera = PsEye()
camera.start()

# Waiting until camera ready
while not camera.frame_valid():
    time.sleep(0.05)

print('Camera Running...')


def get_label_value():
    for attempt in range(10):
        if ser.value_valid():
            repeat = False
            label = ser.get_act_value()
            print("got label {}".format(str(label)))
            return (True, label)
        else:
            print("No valid value")
        time.sleep(0.001)
    return (False, None)
     
    
def log_csv(file, img_name, label):
    with open(file, 'a') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerow([img_name, label])
    pass

img_counter = 0

for a in range(num_images):
    rand_increment = random.randint(100,200) 
    rand_rad = rand_increment/4096 * (2*math.pi)
    ser.write(rand_rad)
    time.sleep(1.2)
    (success, label) = get_label_value()
    if success:
        label =label %(2*math.pi)
        img_name = "{}_{}.png".format(time.time(), label)
        camera.save_last_frame(img_file_path, img_name)
        log_csv(csv_file_path, img_name, label)
        img_counter += 1
        print('attenpt {} successful!'.format(a+1))
    else:
        print('attenpt {} FAILED!'.format(a+1))
        
print("{} out of {} attempts successful".format(img_counter, num_images))
camera.end()
ser.end()
print("closing Threads...")

Starting Serial Thread...
Serial Thread running
Camera Running...
got label 0.29
Saving img 1560770634.7248964_0.29.png
attenpt 0 successful!
got label 0.54
Saving img 1560770635.9693425_0.54.png
attenpt 1 successful!
got label 0.69
Saving img 1560770637.210902_0.69.png
attenpt 2 successful!
got label 0.85
Saving img 1560770638.4541426_0.85.png
attenpt 3 successful!
got label 1.14
Saving img 1560770639.7017002_1.14.png
attenpt 4 successful!
5 out of 5 attempts successful
closing Threads...
